In [1]:
import requests
import csv

In [2]:
# Set up GitHub API access
token = "YOUR_GITHUB_TOKEN"
headers = {"Authorization": f"token {token}"}

In [5]:
# Search users in Paris with more than 200 followers
def search_users_in_paris():
    url = "https://api.github.com/search/users"
    location="Paris"
    followers="200"
    params = {"q": f"location:{location} followers:>{followers}", "per_page": 100}
    response = requests.get(url, headers=headers, params=params)
    return response.json().get("items", [])


In [ ]:
# Fetch user details
def get_user_details(username):
    url = f"https://api.github.com/users/{username}"
    response = requests.get(url, headers=headers)
    return response.json()


In [ ]:
# Fetch repositories for each user
def get_user_repos(username):
    url = f"https://api.github.com/users/{username}/repos"
    params = {"per_page": 500, "sort": "pushed"}
    response = requests.get(url, headers=headers, params=params)
    return response.json()


In [ ]:
# Function to clean up company names
def clean_company_name(company):
    if company:
        return company.strip().lstrip('@').upper()
    return company

In [ ]:
# Save users to CSV
def save_users_to_csv(users, filename="users.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(
            file,
            fieldnames=[
                "login",
                "name",
                "company",
                "location",
                "email",
                "hireable",
                "bio",
                "public_repos",
                "followers",
                "following",
                "created_at",
            ],
        )
        writer.writeheader()
        for user in users:
            writer.writerow(user)

In [ ]:
# Save repositories to CSV
def save_repos_to_csv(repos, filename="repositories.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(
            file,
            fieldnames=[
                "login",
                "full_name",
                "created_at",
                "stargazers_count",
                "watchers_count",
                "language",
                "has_projects",
                "has_wiki",
                "license_name",
            ],
        )
        writer.writeheader()
        for repo in repos:
            writer.writerow(repo)

In [ ]:
# Main function
def main():
    users = search_users_in_paris()
    user_data = []
    repo_data = []

    for user in users:
        details = get_user_details(user["login"])
        user_data.append(
            {
                "login": details["login"],
                "name": details.get("name", ""),
                # Use an empty string if company is None
                "company": clean_company_name(
                    details.get("company", "") or ""
                ),  # Handle NoneType
                "location": details.get("location", ""),
                "email": details.get("email", ""),
                "hireable": "true" if details.get("hireable") is True else "false",
                "bio": details.get("bio", ""),
                "public_repos": details.get("public_repos", 0),
                "followers": details.get("followers", 0),
                "following": details.get("following", 0),
                "created_at": details.get("created_at", ""),
            }
        )

        # Fetch repositories for this user
        repos = get_user_repos(user["login"])
        for repo in repos:
            repo_data.append(
                {
                    "login": user["login"],
                    "full_name": repo["full_name"],
                    "created_at": repo["created_at"],
                    "stargazers_count": repo["stargazers_count"],
                    "watchers_count": repo["watchers_count"],
                    "language": repo.get("language", ""),
                    "has_projects": (
                        "true" if repo["has_projects"] else "false"
                    ),  # Handle booleans
                    "has_wiki": (
                        "true" if repo["has_wiki"] else "false"
                    ),  # Handle booleans
                    "license_name": (
                        repo.get("license", {}).get("key", "")
                        if repo.get("license")
                        else ""
                    ),  # Handle NoneType
                }
            )

    # Save data to CSV
    save_users_to_csv(user_data)
    save_repos_to_csv(repo_data)

In [ ]:
main()